#Рекомендательная система на основе текстовых данных

## Предобработка данных, лемматизация, удаление стоп-слов и пунктуации

In [53]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack


In [ ]:
data = pd.read_parquet("https://huggingface.co/datasets/d0rj/povarenok_recipes_detail/resolve/main/data/train-00001-of-00002-8ce75ff1e8c43597.parquet")
data.head(3)
#Импортируем датасет

,title,page_url,main_image_url,description,creation_datetime,views,kroshki,ingredients,cooking_time,portions_count,cooking_steps,cooking_steps_images_urls,cooking_additional_images_urls,purposes,tags,tastes,nae_value,user_photos_urls,recipe_variants_urls
0,Мини-кутабы,https://www.povarenok.ru/recipes/show/92442/,https://www.povarenok.ru/data/cache/2014jul/24...,Весной и в начале лета практически все с больш...,2014-07-24T09:19:00,16290,"[Выпечка, Изделия из теста, Другие изделия]","[{'count': None, 'name': 'Мука пшеничная'}, {'...",30 минут,4,"[Смешиваем просеянную муку и воду, солим. Заме...",[https://www.povarenok.ru/data/cache/2014jul/2...,[/data/cache/2014jul/24/11/807097_10031nothumb...,"[Для детей, На завтрак, Конкурсные рецепты, Ко...",[завтрак],"[свежий, сочный]","{'100 г блюда': {'carb': 21.4, 'fats': 10.9, '...",[https://www.povarenok.ru/data/cache/2016apr/0...,[https://www.povarenok.ru/recipes/show/105939/...
1,Куриные бедрышки с овощами и прованскими травами,https://www.povarenok.ru/recipes/show/92444/,https://www.povarenok.ru/data/cache/2014jul/24...,Захотелось рецепт оформить роликом. Делала нес...,2014-07-24T11:21:00,2827,[Горячие блюда],"[{'count': '6 шт', 'name': 'Бедро куриное'}, {...",35 минут,3,[],[],[],"[На обед, На второе, На праздничный стол, 23 ф...","[ужин, обед, второе, курица, картофель]","[куриный, пряный, ароматный, сочный]","{'100 г блюда': {'carb': 6.9, 'fats': 6.0, 'kc...",[],[https://www.povarenok.ru/recipes/show/107223/...
2,Хлеб на яблочном соке,https://www.povarenok.ru/recipes/show/92443/,https://www.povarenok.ru/data/cache/2014jul/21...,"Вы не пробовали хлебушек на соке? Мягкий, пуши...",2014-07-24T09:32:00,5697,"[Выпечка, Изделия из теста, Домашний хлеб]","[{'count': '160 мл', 'name': 'Сок свежевыжатый...",110 минут,None,[Разводим дрожжи в теплой водичке и даем им по...,[https://www.povarenok.ru/data/cache/2014jul/2...,[/data/cache/2014jul/24/41/807009_54639nothumb...,"[Для детей, Конкурсные рецепты, Конкурс ""Больш...","[хлеб, дрожжи]","[нежный, ароматный]","{'100 г блюда': {'carb': 44.1, 'fats': 9.9, 'k...",[],"[https://www.povarenok.ru/recipes/show/73982/,..."


In [ ]:
data_mini = data.iloc[:20000]
# В дальнейшем мы столкнемся с тем, что матрица tf-idf будет занимать очень много места на ОЗУ, и когда мы будем считать косинусные расстояния, произойдет переполнение памяти. Поэтому ограничимся подвыборкой в первых 20к рецептов. Думаю, нам будет это достаточно

In [ ]:
data_mini.shape

(20000, 19)

In [ ]:
data.columns
# Посмотрим, какие есть признаки. Помимо названия, описания и ингредиентов есть еще и ссылки на картинки, время готовки, шаги готовки и так далее. По условию задачи нужно ограничиться только описанием и ингридиентами

Index(['title', 'page_url', 'main_image_url', 'description',
       'creation_datetime', 'views', 'kroshki', 'ingredients', 'cooking_time',
       'portions_count', 'cooking_steps', 'cooking_steps_images_urls',
       'cooking_additional_images_urls', 'purposes', 'tags', 'tastes',
       'nae_value', 'user_photos_urls', 'recipe_variants_urls'],
      dtype='object')

In [ ]:
dataframe = data_mini[['description', 'ingredients']]

In [ ]:
dataframe.isnull().sum()
# проверим, есть ли пропуски

,0
description,0
ingredients,0


In [ ]:
data_mini['title'].value_counts()
# Как видим, в наших рецептах есть повторения названий. Не исключаем, что это могут быть и дубликаты, но, скорее всего, просто разные рецепты для одного и того же блюда. Сохраним как есть

,count
title,
Овсяное печенье,13
Пасхальный кулич,8
Яблочный пирог,7
"Салат ""Свежесть""",6
Закуска из баклажанов,6
...,...
Салат со свеклой по-армянски,1
Картофельные сундучки,1
"Творожное печенье ""Ушки""",1


In [54]:
!pip install nltk
import nltk

In [ ]:
dataframe['ingredients'][0]

array([{'count': None, 'name': 'Мука пшеничная'},
       {'count': '100 мл', 'name': 'Вода'},
       {'count': '2 ч. л.', 'name': 'Соль'},
       {'count': '1 пуч.', 'name': 'Зелень'},
       {'count': '150 г', 'name': 'Сыр полутвердый'},
       {'count': '100 г', 'name': 'Творог'},
       {'count': '2 ст. л.', 'name': 'Орехи грецкие'},
       {'count': '2 шт', 'name': 'Яйцо куриное'},
       {'count': '1/2 ч. л.', 'name': 'Перец черный'},
       {'count': '150 г', 'name': 'Сметана'}], dtype=object)

In [ ]:
data1 = dataframe.copy()

In [ ]:
for i in range(dataframe.shape[0]):
  ing = []
  for l in range(len(dataframe['ingredients'][i])):
    ing.append(dataframe['ingredients'][i][l]['name'].lower())
  data1.loc[i, 'ingredients'] = ' '.join(ing)

In [ ]:
data1
# Перевели все ингредиенты к строки

,description,ingredients
0,Весной и в начале лета практически все с больш...,мука пшеничная вода соль зелень сыр полутверды...
1,Захотелось рецепт оформить роликом. Делала нес...,бедро куриное картофель помидоры черри лук кра...
2,"Вы не пробовали хлебушек на соке? Мягкий, пуши...",сок свежевыжатый соль мука пшеничная вода дрож...
3,Этот коктейль вовсе не для детей! Вкусно и хме...,сок свежевыжатый вино белое полусладкое мороже...
4,Вкусный быстрый гороховый суп с копченой груди...,вода горох грудинка картофель морковь лук белы...
...,...,...
19995,Omelette Lorraine. Рецепт быстрый и простой. В...,яйцо куриное печень куриная грибы масло сливоч...
19996,Ои собаги - это кимчхи из огурцов. Это распрос...,огурец морковь чеснок лук репчатый соус соль с...
19997,"Прекрасный нежнейший кекс, очень простой, но о...",яйцо куриное кефир сахар масло сливочное мука ...
19998,"Нарядные, аппетитные сладкие ""бабочки"" с начи...",мука пшеничная дрожжи сахар яйцо куриное молок...


In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('russian'))

def remove_stopwords(text):
    tokens = word_tokenize(text, language='russian')
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word not in string.punctuation]
    return ' '.join(filtered_tokens)


data1['description'] = data1['description'].apply(remove_stopwords)

In [ ]:
data1
# Описание привели к нижнему регистру, удалили стоп-слова и пунктуацию. Для ингредиентов это не делаем, предполагаем что там нет стоп-слов.

,description,ingredients
0,весной начале лета практически большим удоволь...,мука пшеничная вода соль зелень сыр полутверды...
1,захотелось рецепт оформить роликом делала неск...,бедро куриное картофель помидоры черри лук кра...
2,пробовали хлебушек соке мягкий пушистый аромат...,сок свежевыжатый соль мука пшеничная вода дрож...
3,коктейль вовсе детей вкусно хмельно ароматно у...,сок свежевыжатый вино белое полусладкое мороже...
4,вкусный быстрый гороховый суп копченой грудинк...,вода горох грудинка картофель морковь лук белы...
...,...,...
19995,omelette lorraine рецепт быстрый простой вкусн...,яйцо куриное печень куриная грибы масло сливоч...
19996,ои собаги это кимчхи огурцов это распространен...,огурец морковь чеснок лук репчатый соус соль с...
19997,прекрасный нежнейший кекс очень простой оправд...,яйцо куриное кефир сахар масло сливочное мука ...
19998,нарядные аппетитные сладкие `` бабочки '' начи...,мука пшеничная дрожжи сахар яйцо куриное молок...


In [ ]:
stemmer = SnowballStemmer("russian")

def stem(text):
  tokens = word_tokenize(text)
  lemmatized_words = [stemmer.stem(word) for word in tokens]
  return ' '.join(lemmatized_words)

data1['description'] = data1['description'].apply(stem)
data1

,description,ingredients
0,весн начал лет практическ больш удовольств ед ...,мука пшеничная вода соль зелень сыр полутверды...
1,захотел рецепт оформ ролик дела нескольк месяц...,бедро куриное картофель помидоры черри лук кра...
2,пробова хлебушек сок мягк пушист ароматн запах...,сок свежевыжатый соль мука пшеничная вода дрож...
3,коктейл вовс дет вкусн хмельн ароматн удачн со...,сок свежевыжатый вино белое полусладкое мороже...
4,вкусн быстр горохов суп копчен грудинк отличн ...,вода горох грудинка картофель морковь лук белы...
...,...,...
19995,omelette lorraine рецепт быстр прост вкусн фар...,яйцо куриное печень куриная грибы масло сливоч...
19996,о собаг эт кимчх огурц эт распространен корейс...,огурец морковь чеснок лук репчатый соус соль с...
19997,прекрасн нежн кекс очен прост оправда рецепт,яйцо куриное кефир сахар масло сливочное мука ...
19998,нарядн аппетитн сладк `` бабочк `` начинк халв...,мука пшеничная дрожжи сахар яйцо куриное молок...


In [ ]:
data1['ingredients'] = data1['ingredients'].apply(stem)
data1
# Провели лемматизацию двух признаков. Привели к леммам, чтобы было легче работать в дальнейшем

,description,ingredients
0,весн начал лет практическ больш удовольств ед ...,мук пшеничн вод сол зелен сыр полутверд творог...
1,захотел рецепт оформ ролик дела нескольк месяц...,бедр курин картофел помидор черр лук красн мор...
2,пробова хлебушек сок мягк пушист ароматн запах...,сок свежевыжат сол мук пшеничн вод дрожж масл ...
3,коктейл вовс дет вкусн хмельн ароматн удачн со...,сок свежевыжат вин бел полусладк морожен кориц...
4,вкусн быстр горохов суп копчен грудинк отличн ...,вод горох грудинк картофел морков лук бел масл...
...,...,...
19995,omelette lorraine рецепт быстр прост вкусн фар...,яйц курин печен курин гриб масл сливочн мук пш...
19996,о собаг эт кимчх огурц эт распространен корейс...,огурец морков чеснок лук репчат соус сол сахар...
19997,прекрасн нежн кекс очен прост оправда рецепт,яйц курин кефир сахар масл сливочн мук пшеничн...
19998,нарядн аппетитн сладк `` бабочк `` начинк халв...,мук пшеничн дрожж сахар яйц курин молок масл с...


## TF-IDF

In [ ]:
tfidf = TfidfVectorizer()
tfidf1 = tfidf.fit_transform(data1['description'])
# Посчитаем для каждого вектора свои значения для колонки описания

tfidf = TfidfVectorizer()
tfidf2 = tfidf.fit_transform(data1['ingredients'])
# Аналогично для ингредиентов

tfidf_all = hstack((tfidf1, tfidf2))
data_tfidf = pd.DataFrame(tfidf_all.toarray())
data_tfidf
# Соединим обе таблицы данных.

# Почему сделали именно так? Нам важно, чтобы алгоритм видел как описание, так и ингредиенты. Если бы мы сначала объединили наши слова из двух признаков в одно пространство, мы бы потеряли важную информацию про ингредиенты. Потому что в описании картошка могла встретиться только 1 раз и в ингредиентах могло встретиться, но вес в ингредиентах значительно больше.

,0,1,2,3,4,5,6,7,8,9,...,21711,21712,21713,21714,21715,21716,21717,21718,21719,21720
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.155108,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115161,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134403,0.0,0.0
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.213511,0.0,0.0
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.302869,0.0,0.0


In [ ]:
tfidf1.shape

(20000, 20943)

In [ ]:
tfidf2.shape

(20000, 778)

In [ ]:
svd = TruncatedSVD(n_components=200)  # Оставляем 200 компонент. Далее чтобы посчитать матрицу косинусов, нам понадобится очень много памяти. Хранить 20.000 слов это очень дорого, достаточно найти 200 компонент, которые будут самыми важными и спроецировать всю матрицу на них.
tfidf_reduced = svd.fit_transform(data_tfidf)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(tfidf_reduced, tfidf_reduced)

In [ ]:
data_mini_train = data_mini[['title', 'description', 'ingredients']]
data_mini_train
# Посчитали косинусные расстояния. Также давайте для удобства интерпретации алгоритма заведем новый датафрейм, в который добавим еще и названия блюд.

,title,description,ingredients
0,Мини-кутабы,Весной и в начале лета практически все с больш...,"[{'count': None, 'name': 'Мука пшеничная'}, {'..."
1,Куриные бедрышки с овощами и прованскими травами,Захотелось рецепт оформить роликом. Делала нес...,"[{'count': '6 шт', 'name': 'Бедро куриное'}, {..."
2,Хлеб на яблочном соке,"Вы не пробовали хлебушек на соке? Мягкий, пуши...","[{'count': '160 мл', 'name': 'Сок свежевыжатый..."
3,"Коктейль ""Застенчивая дыня""",Этот коктейль вовсе не для детей! Вкусно и хме...,"[{'count': None, 'name': 'Сок свежевыжатый'}, ..."
4,Гороховый суп с копченой грудинкой,Вкусный быстрый гороховый суп с копченой груди...,"[{'count': '2 л', 'name': 'Вода'}, {'count': '..."
...,...,...,...
19995,Омлет лотарингский,Omelette Lorraine. Рецепт быстрый и простой. В...,"[{'count': '4 шт', 'name': 'Яйцо куриное'}, {'..."
19996,"Кимчи из огурцов ""Ои Собаги""",Ои собаги - это кимчхи из огурцов. Это распрос...,"[{'count': '10 шт', 'name': 'Огурец'}, {'count..."
19997,Нежный кекс на кефире,"Прекрасный нежнейший кекс, очень простой, но о...","[{'count': '3 шт', 'name': 'Яйцо куриное'}, {'..."
19998,"Булочки с халвой ""Бабочки""","Нарядные, аппетитные сладкие ""бабочки"" с начи...","[{'count': None, 'name': 'Мука пшеничная'}, {'..."


In [ ]:
ind = pd.Series(data_mini_train.index, index=data_mini_train['title'])

In [ ]:
# Проверим качество работы алгоритма. Например, возьмем Салат с редькой

recept = ind["Салат с редькой"]

similarity_scores = pd.DataFrame(cosine_sim[recept], columns=["score"])
similarity_scores

,score
0,0.029993
1,0.063673
2,0.003810
3,0.005225
4,0.163411
...,...
19995,-0.002531
19996,0.260449
19997,0.015839
19998,0.110143


In [ ]:
# Чтобы посчитать топ-5 похожих блюд, нам нужно посмотреть на вектор значений матрицы косинусных расстояний, взять 1-6 объект (0 не берем, потому что там стоит значение 1 - это диагональный элемент матрицы). Получаем индексы топ-5 похожих блюд.
same_dish = similarity_scores.sort_values("score", ascending=False)[1:6].index
print(same_dish)

Index([9304, 11191, 10433, 8288, 2374], dtype='int64')


In [ ]:
data_mini_train.iloc[same_dish]
# Посмотрим, какие блюда похожи для нашего Салата с редькой

,title,description,ingredients
9304,Салат из капусты с огурцами,Капуста в нашем семейном рационе - вещь постоя...,"[{'count': 'Капустa', 'name': 'Капуста белокоч..."
11191,Овощной салат с постным майонезом из нута,Сам по себе этот салат чем-то похож на винегре...,"[{'count': '1 шт', 'name': 'Морковь'}, {'count..."
10433,"Острый капустный салат ""А-ля Юность""","Странно, что на сайте нет салата ""Юность"". Обя...","[{'count': None, 'name': 'Капуста белокочанная..."
8288,"Салат ""Четыре капусты""","Очень полезный салат, который чистит организм,...","[{'count': None, 'name': 'Капуста белокочанная..."
2374,Салат из капусты и моркови с лимонно-медовой з...,"Легкий, свежий, полезный салат очищения. Напом...","[{'count': None, 'name': 'Капуста белокочанная..."


## Тестирование алгоритма

In [ ]:
# Возьмем 10 блюд из нашего датасета. Для них попробуем посмотреть похожие блюда. Метрика, по которой мы будем оценивать схожесть - среднее косинусное расстояние.
test_dishes = data_mini_train['title'].sample(n=10, random_state=42)
test_dishes

,title
10650,Веер из баклажана с куриным рагу
2041,Шоколадный клафути с вишней
8668,"Блинчики ""Черный лес"""
1114,Английская каша
13902,Забытый салат из крабовых палочек
11963,Говядина тушеная в пряном соусе
11072,Треска по-каталонски
3002,"Печёночные ""розы"" с клюквенным соусом D’arbo"
19771,"Салат ""На радость гостям"""
8115,Чечевичный суп с охотничьими колбасками


In [ ]:
average = []
for dish in test_dishes:
  recept = ind[dish]
  if isinstance(recept, pd.Series):
    recept =  recept.iloc[0] # Бывает ситуация, что одинаковые названия идут для двух разных индексов. В таком случае мы берем первое блюдо с таким индексом
  similarity_scores = pd.DataFrame(cosine_sim[recept], columns=["score"])
  same = similarity_scores.sort_values("score", ascending=False)[1:6].index

  print("Вы выбрали:", dish)
  print()
  print('Что мы готовы порекомендовать:')
  print(data_mini_train['title'].iloc[same])
  mean_score = similarity_scores.iloc[same].mean()

  print(mean_score)
  print('---------------------------------')
  average.append(mean_score)

print('Average for 10:', np.array(average).mean())

Вы выбрали: Веер из баклажана с куриным рагу

Что мы готовы порекомендовать:
8392                                   Сэндвичицца
12628    Пенне ригате с фрикадельками и пармезаном
3986                   Цветы цуккини фаршированные
12952                      Тефтели в овощном соусе
1750                       Рататоли с куриным филе
Name: title, dtype: object
score    0.610015
dtype: float64
---------------------------------
Вы выбрали: Шоколадный клафути с вишней

Что мы готовы порекомендовать:
8069                   Шоколадный клафути с вишней
5027                     Десерт вишнево-шоколадный
16260                   Шоколадный пудинг с вишней
2302                       Шоколадный блинный торт
16190    Шоколадные вареники с вишней "Чёрный лес"
Name: title, dtype: object
score    0.687161
dtype: float64
---------------------------------
Вы выбрали: Блинчики "Черный лес"

Что мы готовы порекомендовать:
7478      Ягодный десерт в шоколаде
7054              Пирожные "Ёлочки"
13537          

In [ ]:
# Среднее косинусное расстояние дле нашего алгоритма на 10 тестовых блюдах - 0.65. Если почитать названия рецептов, мы достаточно хорошо рекомендуем похожие блюда.

In [ ]:
# Итог работы: Провели предобработку данных, использовать tf-idf для векторизации слов. посчитали для каждого ингредиента косинусные расстояния и создали алгоритм рекомендации 5 похожих блюд